In [4]:
# import standard dependencies
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt
import uuid # to generate unique image names

# import tensorflow dependencies - Functional API. passing through 2 images at same time and measure the similarity. if similar = 1, if diff = 0
from tensorflow.keras.models import Model #
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

In [6]:
# setup paths
POS_PATH = os.path.join('data', 'positive') #joins dif directories together to be able to form a full file path. positive verification images
NEG_PATH = os.path.join('data', 'negative') #negative verification image
ANC_PATH = os.path.join('data', 'anchor')
